In [ ]:
import pandas as pd

# put data into Pandas DataFrame
df = pd.read_csv('ifr.csv')

# keep only columns "Date", "Time", and "feed"
df = df[['Date', 'Time', 'feed']]

# convert to DateTime
df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df.set_index('datetime', inplace=True)

# delete Date and Time columns
df = df.drop(columns=['Date', 'Time'])

# drop rows with NaN
df = df.dropna()

# aggregate by summing feed for each day
df_day = df.resample('D').sum()

# find minimum daily feed after January 17, 2023
min_after = df_day[df_day.index >= '2023-01-17']['feed'].min()

# find earlier values lower than this minimum
lower_values = df_day[df_day['feed'] < min_after]

# find datetime of maximum of lower_values
cutoff = lower_values['feed'].idxmax()

# aggregate by summing feed for each 4 hours
df = df.resample('4H').sum()

# get rid of data before cutoff
df_cut = df[df.index >= cutoff + pd.Timedelta('1D')]

In [ ]:
# print the DataFrame
print(df)
print(df_day)
print(min_after)
print(lower_values)
print(cutoff)
print(cutoff  + pd.Timedelta('1D'))
print(df_cut)

In [ ]:
from statsmodels.tsa.forecasting.theta import ThetaModel
import matplotlib.pyplot as plt

# fit the model
model = ThetaModel(df)
model_results = model.fit()

# plot a prediction
model_results.plot_predict(100, in_sample=True, alpha=None)

# add a title
plt.title('Infant Feeding Record')

# add a legend
plt.legend(['Observed', 'Predicted'])

# add a label to the x-axis
plt.xlabel('Date')

# add a label to the y-axis
plt.ylabel('Amount (mL)')

# make the plot twice as wide
plt.gcf().set_size_inches(12, 6)

# add horizontal grid lines
plt.grid(axis='y')

# change x-axis to days elapsed

# show the plot
plt.show()

# do the same for the cut data
model_cut = ThetaModel(df_cut)
model_cut_results = model_cut.fit()
model_cut_results.plot_predict(100, in_sample=True, alpha=None)
plt.title('Infant Feeding Record')
plt.legend(['Observed', 'Predicted'])
plt.xlabel('Date')
plt.ylabel('Amount (mL)')
plt.gcf().set_size_inches(12, 6)
plt.grid(axis='y')
plt.show()